# Ceneo Scraper

## Biblioteki

In [99]:
import requests
from bs4 import BeautifulSoup

## Wysłanie do serwera żądania dostępu do zasobu

In [100]:
product_id = '94103927'
url = f'https://www.ceneo.pl/{product_id}#tab=reviews'
response = requests.get(url)

In [101]:
page_dom = BeautifulSoup(response.text, "html.parser")
# opinions = page_dom.select_one('div.js_product-reviews')

# # opinion = page_dom.select_one('div.js_product-reviews')
# opinion = page_dom.select_one('div.js_product-reviews')

opinions = page_dom.select('div.user-post')
opinion = opinions.pop()
# opinion = page_dom.select('div.user-post:nth-of-type(4)')[0]
# opinion = opinions[3]


## Ekstracja składowych pojedynczej opinii

|Składowa|Selektor|Zmienna|
|--------|--------|-------|
|id_opinii|["data-entry-id"]|opinion_id|
|autor|span.user-post__author-name|author|
|rekomendacja|span.user-post__author-recomendation > em|recommendation|
|gwiazdki|span.user-post__score-count|rating|
|treść|div.user-post__text|content|
|lista zalet|div.review-feature__title--positives ~ div.review-feature__item|pros|
|lista wad|div.review-feature__title--negatives ~ div.review-feature__item|cons|
|dla ilu przydatna|button.vote-yes > span|useful|
|dla ilu nieprzydatna|button.vote-no > span|useless|
|data wystawienia|span.user-post__published > time:nth-child(1)["datetime"]|publish_date|
|data zakupu|span.user-post__published > time:nth-child(2)["datetime"]|purchase_date|


In [102]:
opinion_id = opinion.get('data-entry-id')
author = opinion.select_one('span.user-post__author-name').string
recommendation = opinion.select_one('span.user-post__author-recomendation > em').string
rating = opinion.select_one('span.user-post__score-count').string
content = opinion.select_one('div.user-post__text').string
pros = []
cons = []

if opinion.select_one('div.review-feature__title--positives ~ div.review-feature__item'):
    pros = [x.string for x in opinion.select('div.review-feature__title--positives ~ div.review-feature__item')]
if opinion.select_one('div.review-feature__title--negatives ~ div.review-feature__item'):
    cons = [x.string for x in opinion.select('div.review-feature__title--negatives ~ div.review-feature__item')]

useful = opinion.select_one('button.vote-yes > span').string
useless = opinion.select_one('button.vote-no > span').string
publish_date = opinion.select_one('span.user-post__published > time:nth-child(1)').get('datetime')
purchase_date = opinion.select_one('span.user-post__published > time:nth-child(2)').get('datetime')

print(opinion_id)
print(author)
print(recommendation)
print(rating)
print(content)
print(pros)
print(cons)
print(useful)
print(useless)
print(publish_date)
print(purchase_date)

13699872

s...1
Polecam
5/5
Polecam. Mam taki w pracy, pracuje kucharzem. 
[]
[]
0
0
2021-01-02 19:11:20
2020-12-02 18:02:26
